## Modelo 1 con DataFrame sin nulos
### Vamos a comenzar realizando un árbol de desición

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [ ]:
df_properties = pd.read_csv("datasets/properties_train_wn.csv")
df_properties

In [ ]:
#eliminamos columnas que no vamos a utilizar
df_properties.drop(['Unnamed: 0','price'], axis=1, inplace=True)
df_properties

In [ ]:
plt.figure(figsize=(10,10))
sns.heatmap(df_properties.corr(method='spearman'), annot=True, fmt='.1g', cmap='coolwarm')


In [ ]:
#Determinamos las columnas que vamos a utilizar para ver su correlación con el precio
columnas = ['sqfeet','beds','baths','cats_allowed','dogs_allowed','smoking_allowed','wheelchair_access','electric_vehicle_charge','comes_furnished']



In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
 

In [ ]:
# Instanciamos un objeto de la clase DecisionTreeClassifier

clf = DecisionTreeClassifier(max_depth = 3, random_state = 42)

In [ ]:
#importamos el archivo de test
df_properties_test = pd.read_parquet("datasets/test.parquet")

In [ ]:
# Entrenamos el modelo
X = df_properties[columnas]  # Denotamos X con mayúscula ya que incluye más de un atributo
y = df_properties.category_price # Etiqueta a predecir
clf.fit(X.values,y.values) #entrenamos con train


In [ ]:
# Predecimos

y_pred = clf.predict(X.values)

In [ ]:
accuracy = accuracy_score(y, y_pred)
print("La precisión del modelo es: ", accuracy)

In [ ]:
#Hacemos el modelo con el archivo de test
X_train = df_properties[columnas]
y_train = df_properties['category_price']

X_test = df_properties_test[columnas]

clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

df_properties_test['category_price']= y_pred
y_test = df_properties_test['category_price']

In [ ]:
df_properties_test['category_price'].to_csv('datasets/YamiGaliano.csv', index=False) 

## Modelo 1.2 remplazando cualitativos en cuantitativos

Vamos a pasar variables cualitativas a cuantitativas: type, laundry_options y parking_options. Region y state los vamos a analizar. 

In [ ]:
#Vamos a eliminar primero columnas de mas en test trabajar con datos mas pequeños
df_properties_test.drop(['url', 'region_url','image_url','description'], axis=1, inplace=True)

In [ ]:
df_properties_test.shape

In [ ]:
df_properties.shape

In [ ]:
df_properties_test.isna().sum()

Type

In [ ]:
#Validamos los valores unicos y creamos un diccionario con los valores
df_properties.type.unique()

dic_type = {'house':1, 'apartment':2, 'duplex':3, 'townhouse':4, 'in-law':5, 'condo':6,
       'cottage/cabin':7, 'manufactured':8, 'flat':9, 'loft':10, 'assisted living':11,
       'land':12}


In [ ]:
#Utilizamos la función map() para reemplazar los valores de la variable cualitativa con el diccionario
df_properties["type_1"] = df_properties["type"].map(dic_type)

#Hacemos lo mismo en el df de test
df_properties_test["type_1"] = df_properties_test["type"].map(dic_type)

df_properties_test.head(5)

Laundry_options

In [ ]:
#Validamos los valores unicos y creamos un diccionario con los valores
df_properties.laundry_options.unique()

dic_laundry = {'w/d in unit':1, 'w/d hookups':2, 'laundry on site':3, 'laundry in bldg':4, 'no laundry on site':5}

In [ ]:
#Utilizamos la función map() para reemplazar los valores de la variable cualitativa con el diccionario
df_properties["laundry"] = df_properties["laundry_options"].map(dic_laundry)

#Hacemos lo mismo en el df de test
df_properties_test["laundry"] = df_properties_test["laundry_options"].map(dic_laundry)
df_properties_test.head(5)
#Observación: en df_properties_test la columna quedo como float debido a que tenemos los nulos o None en la columna. 

In [ ]:
#Asignamos un 0 a los nulos
df_properties_test['laundry'] = df_properties_test['laundry'].replace(np.nan, 0)
df_properties_test.isna().sum()

Parking_options

In [ ]:
#Validamos los valores unicos y creamos un diccionario con los valores
df_properties.parking_options.unique()

dic_parking = {'detached garage':1, 'off-street parking':2, 'attached garage':3,'carport':4, 'no parking':5, 'street parking':6, 'valet parking':7}

In [ ]:
#Utilizamos la función map() para reemplazar los valores de la variable cualitativa con el diccionario
df_properties["parking"] = df_properties["parking_options"].map(dic_parking)

#Hacemos lo mismo en el df de test
df_properties_test["parking"] = df_properties_test["parking_options"].map(dic_parking)
df_properties_test.head(5)
#Observación: en df_properties_test la columna quedo como float debido a que tenemos los nulos o None en la columna. 

In [ ]:
#Asignamos un 0 a los nulos
df_properties_test['parking'] = df_properties_test['parking'].replace(np.nan, 0)
df_properties_test.isna().sum()

State

In [ ]:
# Contar la frecuencia de cada valor en la columna
value_counts = df_properties["state"].value_counts()
# Asignar un valor numérico a cada categoría de acuerdo a su frecuencia
df_properties["state_freq"] = df_properties["state"].map(value_counts)


#ahora en el test
value_counts_test = df_properties_test["state"].value_counts()
df_properties_test["state_freq"] = df_properties_test["state"].map(value_counts_test)

df_properties_test.head(5)

Graficamos las relaciones

In [ ]:
plt.figure(figsize=(10,10))
sns.heatmap(df_properties.corr(method='spearman'), annot=True, fmt='.1g', cmap='coolwarm')

Eliminamos columnas y creamos de nuevo el modelo árbol

In [ ]:
#Vamos a eliminar primero columnas de mas en test trabajar con datos mas pequeños
#df_properties.drop(['laundry_options', 'parking_options','state'], axis=1, inplace=True)
#df_properties_test.drop(['laundry_options', 'parking_options','state'], axis=1, inplace=True)

#Definimos columnas a utilizar
columnas = ['sqfeet','beds','baths','cats_allowed','dogs_allowed','wheelchair_access','type_1','laundry','state_freq','parking']

In [ ]:
# Instanciamos un objeto de la clase DecisionTreeClassifier

clf1 = DecisionTreeClassifier(max_depth = 3, random_state = 42)

In [ ]:
# Entrenamos el modelo
X = df_properties[columnas]  # Denotamos X con mayúscula ya que incluye más de un atributo
y = df_properties.category_price # Etiqueta a predecir
clf1.fit(X.values,y.values) #entrenamos con train

# Predecimos
y_pred = clf1.predict(X.values)

accuracy = accuracy_score(y, y_pred)
print("La precisión del modelo es: ", accuracy)

In [ ]:
#Hacemos el modelo con el archivo de test
X_train = df_properties[columnas]
y_train = df_properties['category_price']

X_test = df_properties_test[columnas]

clf1 = DecisionTreeClassifier()
clf1.fit(X_train, y_train)

y_pred = clf1.predict(X_test)

df_properties_test['pred']= y_pred

In [ ]:
df_properties_test['pred'].to_csv('datasets/YamiGaliano.csv', index=False) 

## Modelo 1.3 reduciendo columnas

In [ ]:
#Vamos a eliminar primero columnas de mas en test trabajar con datos mas pequeños
#df_properties.drop(['laundry_options', 'parking_options','state'], axis=1, inplace=True)
#df_properties_test.drop(['laundry_options', 'parking_options','state'], axis=1, inplace=True)

#Definimos columnas a utilizar
columnas = ['sqfeet','beds','baths']

In [ ]:
# Instanciamos un objeto de la clase DecisionTreeClassifier

clf2 = DecisionTreeClassifier(max_depth = 5, random_state = 42)

In [ ]:
# Entrenamos el modelo
X = df_properties[columnas]  # Denotamos X con mayúscula ya que incluye más de un atributo
y = df_properties.category_price # Etiqueta a predecir
clf2.fit(X.values,y.values) #entrenamos con train

# Predecimos
y_pred = clf2.predict(X.values)

accuracy = accuracy_score(y, y_pred)
print("La precisión del modelo es: ", accuracy)

In [ ]:
#Hacemos el modelo con el archivo de test
X_train = df_properties[columnas]
y_train = df_properties['category_price']

X_test = df_properties_test[columnas]

clf2 = DecisionTreeClassifier()
clf2.fit(X_train, y_train)

y_pred = clf2.predict(X_test)

df_properties_test['pred']= y_pred

In [ ]:
df_properties_test['pred'].to_csv('datasets/YamiGaliano.csv', index=False) 

## Modelo 1.4 vecinos

In [ ]:
columnas = ['sqfeet','beds','baths','cats_allowed','dogs_allowed','wheelchair_access','type_1','laundry','state_freq','parking']

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

# Instanciamos un objeto de la clase KNeighborsClassifier

clf4 = KNeighborsClassifier()


In [ ]:
# Entrenamos el modelo
X = df_properties[columnas]  # Denotamos X con mayúscula ya que incluye más de un atributo
y = df_properties.category_price # Etiqueta a predecir

clf4.fit(X.values,y.values)
# Predecimos

y_pred = clf4.predict(X.values)
# Evaluamos

accuracy = accuracy_score(y, y_pred)
print("La precisión del modelo es: ", accuracy)

In [ ]:
#Hacemos el modelo con el archivo de test
X_train = df_properties[columnas]
y_train = df_properties['category_price']

X_test = df_properties_test[columnas]

clf2 = DecisionTreeClassifier()
clf2.fit(X_train, y_train)

y_pred = clf2.predict(X_test)

df_properties_test['pred']= y_pred

In [ ]:
df_properties_test['pred'].to_csv('datasets/YamiGaliano.csv', index=False) 